# Patch level cloud coverage

This notebook obtains patch level (32x32 subset) cloud cover percentages from the Scene classification mask tied to a Sentinel-2 dataset.

In [14]:
import geopandas as gpd
import pystac_client
import shapely
import stackstac
import torch
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import xarray as xr
import rasterio
import rioxarray  # noqa: F401
import pyarrow as pa
import pickle
import lancedb
import glob
from pathlib import Path
from shapely.geometry import Point, Polygon, box
from rasterio.enums import Resampling

pd.set_option('display.max_colwidth', None)

BAND_GROUPS = {
    "rgb": ["red", "green", "blue"],
    "rededge": ["rededge1", "rededge2", "rededge3", "nir08"],
    "nir": [
        "nir",
    ],
    "swir": ["swir16", "swir22"],
    "sar": ["vv", "vh"],
    "scl": ["scl"],
}

# Cloud labels in the SCL band: https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/
SCL_CLOUD_LABELS = [7, 8, 9, 10]

STAC_API = "https://earth-search.aws.element84.com/v1"
COLLECTION = "sentinel-2-l2a"

## Find Sentinel-2 scenes stored as Cloud-Optimized GeoTIFFs

#### Define an area of interest
This is a hotspot area where mining extraction occurs on the island of Fiji. We used this in another tutorial, albeit with a cloud free composite. This will help demonstrate how we can capture clouds for the same region and time frame in the absence of a cloud-free composite.

In [15]:
# sample cluster
bbox_bl = (177.4199,-17.8579)
bbox_tl = (177.4156,-17.6812)
bbox_br = (177.5657,-17.8572)
bbox_tr = (177.5657,-17.6812)

Define spatiotemporal query

In [16]:
# Define area of interest
area_of_interest = shapely.box(xmin=bbox_bl[0], ymin=bbox_bl[1], xmax=bbox_tr[0], ymax=bbox_tr[1])

# Define temporal range
daterange: dict = ["2021-01-01T00:00:00Z", "2021-12-31T23:59:59Z"]

In [17]:
catalog = pystac_client.Client.open(STAC_API)

search = catalog.search(
    collections=[COLLECTION],
    datetime=daterange,
    intersects=area_of_interest.centroid,
    max_items=100,
    query={"eo:cloud_cover": {"lt": 80}},
)

items = search.get_all_items()

print(f"Found {len(items)} items")

Found 80 items


## Download the data
Get the data into a numpy array and visualize the imagery. 

In [ ]:
# Extract coordinate system from first item
epsg = items[0].properties["proj:epsg"]

# Convert point from lon/lat to UTM projection
poidf = gpd.GeoDataFrame(crs="OGC:CRS84", geometry=[area_of_interest.centroid]).to_crs(
    epsg
)
geom = poidf.iloc[0].geometry

# Create bounds of the correct size, the model
# requires 512x512 pixels at 10m resolution.
bounds = (geom.x - 2560, geom.y - 2560, geom.x + 2560, geom.y + 2560)

# Retrieve the pixel values, for the bounding box in
# the target projection. In this example we use only
# the RGB, NIR and SCL band groups.
stack = stackstac.stack(
    items[0],
    bounds=bounds,
    snap_bounds=False,
    epsg=epsg,
    resolution=10,
    dtype="float32",
    rescale=False,
    fill_value=0,
    assets=BAND_GROUPS["rgb"] + BAND_GROUPS["nir"] + BAND_GROUPS["scl"],
    resampling=Resampling.nearest,
)

stack = stack.compute()
print(stack.shape)
assert stack.shape == (1, 5, 512, 512)

stack.sel(band=["red", "green", "blue"]).plot.imshow(
    row="time", rgb="band", vmin=0, vmax=2000
)

### Get the geospatial bounds for the 32x32 windows 
We will use the geospatial bounds of the 32x32 windowed subsets ("chunks") to store the patch level embeddings.

In [ ]:
# Define the chunk size for tiling
chunk_size = {'x': 32, 'y': 32}  # Adjust the chunk size as needed

# Tile the data
ds_chunked = stack.chunk(chunk_size)

# Get the dimensions of the data array
dims = ds_chunked.dims

# Get the geospatial information from the original dataset
geo_info = ds_chunked.attrs

# Iterate over the chunks and compute the geospatial bounds for each chunk
chunk_bounds = {}

# Get the geospatial transform and CRS
transform = ds_chunked.attrs['transform']
crs = ds_chunked.attrs['crs']

for x in range((ds_chunked.sizes['x'] // chunk_size['x']) + 1):
    for y in range((ds_chunked.sizes['y'] // chunk_size['y']) + 1):
        # Compute chunk coordinates
        x_start = x * chunk_size['x']
        y_start = y * chunk_size['y']
        x_end = min(x_start + chunk_size['x'], ds_chunked.sizes['x'])
        y_end = min(y_start + chunk_size['y'], ds_chunked.sizes['y'])
        
        # Compute chunk geospatial bounds
        lon_start, lat_start = transform * (x_start, y_start)
        lon_end, lat_end = transform * (x_end, y_end)
        print(lon_start, lat_start, lon_end, lat_end, x, y)

        # Store chunk bounds
        chunk_bounds[(x, y)] = {
            'lon_start': lon_start, 'lat_start': lat_start,
            'lon_end': lon_end, 'lat_end': lat_end
        }

# Print chunk bounds
#for key, value in chunk_bounds.items():
    #print(f"Chunk {key}: {value}")



In [29]:
# Function to count cloud pixels in a subset
def count_cloud_pixels(subset_scl):
    cloud_pixels_7 = numpy.count_nonzero(subset_scl == SCL_CLOUD_LABELS[0])
    cloud_pixels_8 = numpy.count_nonzero(subset_scl == SCL_CLOUD_LABELS[1])
    cloud_pixels_9 = numpy.count_nonzero(subset_scl == SCL_CLOUD_LABELS[2])
    cloud_pixels_10 = numpy.count_nonzero(subset_scl == SCL_CLOUD_LABELS[3])
    cloud_pixels = cloud_pixels_7 + cloud_pixels_8 + cloud_pixels_9 + cloud_pixels_10
    return cloud_pixels

# Iterate over the chunks and compute the cloud count for each chunk
cloud_counts = {}

for x in range((ds_chunked.sizes['x'] // chunk_size['x']) + 1):
    for y in range((ds_chunked.sizes['y'] // chunk_size['y']) + 1):
        # Compute chunk coordinates
        x_start = x * chunk_size['x']
        y_start = y * chunk_size['y']
        x_end = min(x_start + chunk_size['x'], ds_chunked.sizes['x'])
        y_end = min(y_start + chunk_size['y'], ds_chunked.sizes['y'])
        
        # Extract the subset of the SCL band
        subset_scl = ds_chunked[:,:,5][:, y_start:y_end, x_start:x_end]
        
        # Count the cloud pixels in the subset
        cloud_count = count_cloud_pixels(subset_scl)
        
        # Store the cloud count for this chunk
        cloud_counts[(x, y)] = cloud_count

# Print cloud counts
for key, value in cloud_counts.items():
    if value > 0:
        print(f"Chunk {key}: Cloud Count = {value}")


Chunk (6, 0): Cloud Count = 2
Chunk (7, 0): Cloud Count = 32
Chunk (8, 0): Cloud Count = 16
Chunk (9, 0): Cloud Count = 18
Chunk (10, 0): Cloud Count = 4
